In [ ]:
# plot f/f0 vs log t and omega vs log t (m1 is mass of main star, m2 mass of companion)

# important libs and constants
import plot_utils as p
import functions as fct
import ODE_solver as odes
import numpy as np
import random
G = 6.674e-11  # N*m^2/Kg^2
Msolar = 1.989e30  # kg
Rsolar = 6.957e8  # in m
c = 299792458.0  # m/s

# shit just got real
for m1 in [random.uniform(10*Msolar, 100*Msolar) for _ in range(10)]:
    for m2 in [random.uniform(20*Msolar, 100*Msolar) for _ in range(10)]:
        for rg2 in [0.1]:
            for k in [0.001]:
                for Q in [1e4, 1e5, 1e6, 1e7]:
                    for lifetime in [1000, 5000, 10000, 1e5]:
                        q = m1/m2

                        RWR1 = fct.radius_from_mass(
                            m1/Msolar)  # in solar radii
                        R1 = RWR1*Rsolar  # in m

                        a_max = fct.max_separation(1.36e10, m1, m2)  # in m
                        a_min = fct.min_separation(q, RWR1)  # in solar radii
                        a_min_si = a_min*Rsolar  # in m

                        fmax = fct.gw_frequency(a_max, m1, m2)
                        fmin = fct.gw_frequency(a_min_si, m1, m2)

                        T_TF_max = fct.tidal_friction_timescale(
                            m1, m2, Q, k, a_max, RWR1, fmax)
                        T_TF_min = fct.tidal_friction_timescale(
                            m1, m2, Q, k, a_min_si, RWR1, fmin)

                        tfinal = lifetime*365.25*24*3600  # time in s

                        # checking timescales compared to lifetime

                        T_TF = fct.tidal_friction_timescale(
                            m1, m2, Q, k, a_min_si, RWR1, fmin)/(3600*24*365.25)
                        if T_TF <= lifetime:

                            # and now I die 2.0

                            K1 = (18*k/Q)*(m2*(np.pi**(13/3))*(R1**5)) / \
                                ((G**(5/3))*m1*rg2*(m1+m2)**(5/3))
                            K2 = (3*k/Q)*((m2**2)*(np.pi**3)*(R1**3)) / \
                                (G*m1*rg2*(m1+m2)**2)

                            def dfdt(f, Omega):
                                return K1*(f**(13/3))*(f/2-Omega)

                            def dOmegadt(f, Omega):
                                return K2*(f**3)*(f/2-Omega)

                            for a0 in [2*a_min_si, 4*a_min_si, 6*a_min_si, 8*a_min_si, 10*a_min_si]:
                                f0 = fct.gw_frequency(a0, m1, m2)
                                for Omega0 in [1e-5]:
                                    sols = odes.solve_Radau(
                                        dxdt=dfdt, dydt=dOmegadt, x0=f0, y0=Omega0, t0=0, tfinal=tfinal, x_scale=1, y_scale=1, t_scale=1)

                                    t = sols[0]/(3600*24*365.25)  # in years
                                    f = sols[1]  # in Hz
                                    Omega = sols[2]  # in Hz
                                    # plot f/f0(t)
                                    p.plot2d(t, f/f0, xlabel='t [years]',
                                             ylabel='f/f0', title=f"f/f0 vs t for m1={format(m1/Msolar, '.0e')} M_sun, m2={format(m2/Msolar, '.0e')} M_sun, k={format(k, '.0e')}, Q={format(Q, '.0e')}, a0={format(a0, '.2e')} m, Ω0={format(Omega0, '.2e')} Hz, lifetime={format(lifetime, '.0e')} years, T_TF={format(T_TF, '.0e')} years, f0={format(f0, '.2e')} Hz, f_final={format(f[-1], '.2e')} Hz, Omega_final={format(Omega[-1], '.2e')} Hz", legend='f/f0(t)', colour=None, axis_font='Tahoma', axis_size=12, title_font='Georgia', title_size=16, legend_size=10)
                                    # plot Omega(t)
                                    p.plot2d(t, Omega, xlabel='t [years]',
                                             ylabel='Ω [Hz]', title=f"Ω vs t for m1={format(m1/Msolar, '.0e')} M_sun, m2={format(m2/Msolar, '.0e')} M_sun, k={format(k, '.0e')}, Q={format(Q, '.0e')}, a0={format(a0, '.2e')} m, Ω0={format(Omega0, '.2e')} Hz, lifetime={format(lifetime, '.0e')} years, T_TF={format(T_TF, '.0e')} years, f0={format(f0, '.2e')} Hz, f_final={format(f[-1], '.2e')} Hz, Omega_final={format(Omega[-1], '.2e')} Hz", legend='Ω(t)', colour=None, axis_font='Tahoma', axis_size=12, title_font='Georgia', title_size=16, legend_size=10)